<a href="https://colab.research.google.com/github/dsapandora/DEMUCS/blob/master/DemucsFacebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uso de la libreria DEMUCS para separar pistas de un archivo de audio.

Lo primero que se debe hacer es la instalacion de **Anaconda**, que funcionara como sistema de gestion de liberias para el proyecto. (Esta primera etapa no es necesaria si ya tiene instalado ANACONDA en sus maquinas)
## Descargar el software ananconda.

In [0]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh && bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local


## Configurar el software Anaconda para ser cargado en e el entorno de Google Colab

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

## Verificar si la instalacion ocurrio correctamente